Exercise 07: Undoing temperature
===========================================
<img src="BFnature17440.jpg" style="max-width:45%; float:right; padding-left:30pt">

One of the main ways of probing systems is by measuring absorption spectra,
e.g., measuring how much light is absorbed by the material based on its wavelength.

On the right you find such a spectrum I took from [Spaun et al. (2016)]: the horizontal axis
is the wavelength of the light and the vertical axis is the proportion of the light that
is aborbed by the system.
Note that as we reduce the temperature, top plot to bottom plot, the main thing that
is happening is that the features are getting "sharper" and less blurry. (There are
population effects as well, but we will ignore those for the moment.)

This is because thermal movement causes absorption lines to be less "precise", an effect
known as [Doppler broadening].  If we denote by $A(\omega)$ the observed spectrum at
given temperature $T$, it is related to the "true" spectrum $\rho(\omega)$ at absolute zero
by:
$$
    A(\omega) = \int d\omega'\ K(\omega - \omega')\ \rho(\omega'),
$$
where $K$ is the effect of Doppler broadening.

If we assume a discrete set of observed frequencies $\omega_0, \ldots, \omega_{N-1}$
as well as a discrete set of "hidden" frequencies $\omega'_0, \ldots, \omega'_{K-1}$, then
this equation becomes:
$$
    A(\omega_n) = \sum_{k=0}^{K-1} K(\omega_n, \omega'_k) \rho(\omega'_k) + \epsilon(\omega_n),
$$
where $\epsilon$ is now the inaccuracy in our measurement.  Collecting the spectrums for all
frequencies into a vector, we can write this in a very familiar form:
$$
    \vec A = K \vec\rho + \vec\epsilon.
$$

We would like to "undo" the effect of Doppler broadening to get to the true,
physical resonances $\rho$ of our system.  This simply
amounts to finding $\rho$ for a given $A$ and $K$.  This is nothing but linear
regression, where $A$ is the labels vector, $K$ is the design matrix, and $\rho$ are the parameters we are looking for.

[Spaun et al. (2016)]: https://www.nature.com/articles/nature17440
[Doppler broadening]: https://en.wikipedia.org/wiki/Doppler_broadening

In [ ]:
import numpy as np
import matplotlib.pyplot as pl

The code below loads the broadening kernel into `K`, `omega` holds the set of frequencies.  We have measured three spectrums, which we are going to use for training, validation, and testing, respectively: `A_train`, `A_validate`, and `A_test`.

Below, please 

 1. plot each of these spectra over frequency (you can plot in the same figure).
 2. make a pseudocolor plot (`pl.pcolor_mesh`) of the broadening kernel. Please add a colorbar.

In [ ]:
with np.load("broadening.npz") as _data_file:
    omega = _data_file["w"][:400]
    K = _data_file["K"][:400, :400]
    A_train = _data_file["Atrain"][:400]
    A_validate = _data_file["Avalidate"][:400]
    A_test = _data_file["Atest"][:400]

In [ ]:
# Plot spectra here
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# Plot kernel here
# YOUR CODE HERE
raise NotImplementedError()

Linear model with Gradient descent
---------------------
As a primer for more complicated methods, let us train this model with gradient
descent (GD).

Below you find the code for gradient descent (so that
you don't have to write it yet again).  This code trains the parameters of a
linear model  starting from $\vec\theta = 0$.  Remember, the cost function
of such a model is:
$$
    E(\vec\theta) = \frac 1N \sum_{n=0}^{N-1} E_n(\vec\theta) = \frac 1N \sum_{n=0}^{N-1} | \vec\theta^T \vec x_n - y_n |^2.
$$

Since we have now written enough Gradient Descent solvers, I have provided
both `error_linear`, which computes the cost without regularization, and `gd_step_linear`. This function is a bit special as it performs only a single Gradient descent **step**, taking in the current `theta`, $\theta^{(t)}$, and returning the next time step, $\theta^{(t+1)}$.

In [ ]:
def error_linear(X, y, theta):
    """Return the value of the cost function for a linear model.
    
    Arguments:
      - X:         design matrix
      - y:         labels
      - theta:     parameters
    """
    # Compute value of cost function
    N, K = X.shape
    r = X @ theta - y
    return 1/N * (r @ r)

def gd_step_linear(X, y, theta, eta=0.1):
    """Perform single gradient descent step with a linear model.
    
    Arguments:
      - X:         design matrix (observations, features)
      - y:         labels
      - theta:     parameters to update
      - eta:       learning rate
      
    Return updated parameter
    """
    N, K = X.shape
    
    # Compute gradient of the cost function
    g = 2/N * (X.T @ (X @ theta - y))

    # Perform gradient descent step
    v = -eta * g
    return theta + v

Below, use the above functions to fit `rho` from `A_train` using 200 steps of
Gradient Descent.  Start from `rho` that is simply a vector of zeros and use a
learning rate of `0.1`.

You should create three lists of 201 elements each (one for each iteration $t=0,\ldots,200$):  the list `rhos` contains the current fitted "unblurred" spectrum, `E_train` contains the training errors, and `E_validate` contains the validation errors, which are computed from `A_validate`.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
E_train[:10]

In [ ]:
assert (rhos, E_train, E_validate) is not None
assert len(rhos) == len(E_train) == len(E_validate) == 201
assert rhos[0].shape == omega.shape

np.testing.assert_allclose(rhos[0], 0, atol=1e-7, err_msg="Start with zero")
np.testing.assert_array_less(np.diff(E_train), 0, 
        err_msg="Training error must be strictly decreasing for linear models")
np.testing.assert_array_less(np.diff(E_validate[:20]), 0, 
        err_msg="Validation error in this case decreases in the beginning")

Plot the training error and the validation error over the iteration number.
Use a logarithmic scale for the error.  You also might want to zoom in
a little bit (`pl.ylim`) to see the behaviour at larger iteration number
more clearly.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

Observe that the training and validation error behave similarly
when plotted over GD iteration as they do when plotted over the
number of features. Why?

YOUR ANSWER HERE

Early stopping
------------------
We performed 200 iterations of Gradient Descent, however it
turns out in this case this was wasteful; we could have stopped
our procedure earlier.

From the variables you recorded in your GD, compute the iteration number
where we should have stopped and store it in `iter_opt`.

In [ ]:
# iter_opt = ???
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# Just check that the iteration makes *some* sense...
assert iter_opt > 10 and iter_opt < 150


Now let us analyze how stopping our GD at different iterations influences the shape of our spectra.

Below I have plotted three spectra (elements of `rhos`): at iteration `2`, at iteration `iter_opt`, and at iteration `200`.

In [ ]:
def _plot_one(idx, i, name):
    pl.subplot(idx)
    pl.fill_between(omega, 0, rhos[i])
    pl.ylabel(name)
    pl.ylim(0, 30)
    pl.xlim(0, 10)

_plot_one(311, 2, r'$\rho_2$')
pl.gca().set_xticklabels([])
pl.title("Learned spectra at different gradient descent times/steps")
_plot_one(312, iter_opt, r'$\rho_\mathrm{opt}$')
pl.gca().set_xticklabels([])
_plot_one(313, 200, r'$\rho_{200}$')
pl.xlabel(r'$\omega$')
pl.tight_layout(pad=.1)

Let us summarize our observations:

 1. Why do our spectra get more "spiky" as we add more gradient descent steps?
   
 2. How does early stopping help finding a "good" solution here?

YOUR ANSWER HERE

Stochastic Gradient Descent
----------------------------------------

Finally, we are going to analyze the effect of stochasticity.

When using use SGD, we need to make sure our observations are properly randomized.
For this, I have provided you with a function which takes in some observations
`(X, y)`, and returns them in a shuffled order.

In [ ]:
rng = np.random.default_rng(4711)

def shuffle_observations(X, y):
    """Shuffles observations (useful for stochastic GD)
    
    Use it as follows to shuffle some design matrix X and some associated labels y:
    
        X, y = shuffle_observations(X, y)   
    """
    N, = y.shape
    perm = rng.permutation(N)
    X = X[perm, :]
    y = y[perm]
    return [X, y]

Next, let us code up stochastic gradient descent.

For this, you adapt you `gd_step_linear` to, instead of performing
a single gradient descent step, perform a stochastic gradient descent
epoch.

For this, we first shuffle the observations (I have done that for you).
Then, divide the observations in `X` and `y` into minibatches
of size `M` each.  Then you perform a Gradient Descent step for each
of these minibatches.

**Hints**: you can reshape the arrays `X` and `y` to add a "minibatch"
dimension or alternatively use slicing (e.g., `y[0:10]`) to get the
minibatch.  You can reuse the function `gd_step_linear`.

In [ ]:
rng = np.random.RandomState(4711)

In [ ]:
def sgd_epoch_linear(X, y, theta, M=2, eta=0.1):
    """Perform single stochastic gradient descent epoch with a linear model.
    
    Arguments:
      - X:         design matrix (observations, features)
      - y:         labels
      - theta:     parameters to update
      - M:         size of minibatch
      - eta:       learning rate
    """
    N, K = X.shape
    Nprime = N // M    # number of minibatches
    if N % M != 0:
        raise ValueError(
            "%d observations cannot be divided into minibatches of size %d"
            % (N, M))

    # Shuffle all the observations
    X, y = shuffle_observations(X, y)
    
    # Perform a single stochastic Gradient descent epoch
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# Remove the trailing ; to see the output
sgd_epoch_linear(K, A_train, np.zeros(400), M=10);

In [ ]:
rng = np.random.default_rng(4711)
_rho0 = rng.normal(size=400)

np.testing.assert_allclose(
    sgd_epoch_linear(K, A_train, _rho0, M=400),
    gd_step_linear(K, A_train, _rho0),
    err_msg="SGD for full batch (M=400) should be same as GD")

Adapt your code from above that fits `rho` from `A_train` using 200 steps of
GD to instead use 200 epochs of stochastic gradient descent with a minibatch
size of `M=50` and `eta=0.1`

You should again create three lists of 201 elements each (one for each epoch $t=0,\ldots,200$):  the list `rhos_stoch` contains the current fitted "unblurred" spectrum, `E_train_stoch` contains the training errors, and `E_validate_stoch` contains the validation errors, which are computed from `A_validate`.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

Below, I plot now the errors for the full Gradient Descent (black) and the Stochastic Gradient Descent (red).

In [ ]:
pl.semilogy(E_train, '--b', label='GD: training error')
pl.semilogy(E_validate, '-b', label='GD: validation error ')
pl.plot(E_train_stoch, '--r', label='SGD: training error')
pl.plot(E_validate_stoch, '-r', label='SGD: validation error')
pl.xlabel('iteration')
pl.ylabel('$E/N$')
pl.ylim(3e2, 2e3)
pl.xlim(-1, 201)
pl.legend()

Answer the following questions below:

 1. The training error in SGD is not monotonically falling as in GD. Why?
 
 2. The SGD validation error saturates at a higher level as the GD error. Why?

YOUR ANSWER HERE